In [1]:
from cosmiclib import Compressor, Optimizer

In [2]:
compressor1 = Compressor('Compressor 1', qmin=0.5, qmax=1, wmin=0.5, wmax=1)
compressor2 = Compressor('Compressor 2', qmin=0.5, qmax=0.75, wmin=0.75, wmax=7/8)
my_optimizer = Optimizer(compressors=[compressor1, compressor2])
my_optimizer.setup_problem()
my_optimizer.find_opt(1.5)

{'compressors': {'Compressor 1': {'theta': True, 'q': 0.75, 'w': 0.75},
  'Compressor 2': {'theta': True, 'q': 0.75, 'w': 0.875}},
 'Qtot': 1.5,
 'Wtot': 1.625,
 'Success': True,
 'OptimizationStatus': <OptimizationStatus.OPTIMAL: 0>}